# HOMEWORK 5

In this homework you are going to implement the **Floyd-Steinberg dithering** algorithm. Dithering, in general, means that we are adding noise to the signal (in our case digital image) in order to perceive it better. In other words, by adding the noise the objective quality will be worse but the subjective quality will be better (i.e. the image will "look" better).

The details of FS dithering can be found in this [wiki](https://en.wikipedia.org/wiki/Floyd%E2%80%93Steinberg_dithering) page. In order to implement the dithering, we will implement the following steps:
* Define colour palette
* Quantize the image to obtain the baseline and compute the average quantization error
* Implement FS dithering and compute the average quantization error

You will also have to answer the question at the end of this notebook.

Note: In this homework, you will have the chance to earn some extra points. See the "Bonus" section at the end of the notebook. Good luck!

As always, you are encouraged to use your own images :-)

In [ ]:
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt
from math import sqrt, log10
plt.rcParams['figure.figsize'] = [15, 10]

Let's load the image.

In [ ]:
# Load image
img = cv2.imread('data/kodim23.png')
# Convert it to RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# Plot it
plt.imshow(img)

Let's start with gray tones first.

In [ ]:
# Black, dark gray, light gray, white
colors = np.array([[0, 0, 0],
                   [64, 64, 64],
                   [192, 192, 192],
                   [255, 255, 255]])
colors = colors/255

Using the colour palette, let's quantize the original image.

In [ ]:
# Cast the image to float
img_norm = img
img = img/255

In [ ]:
def closest(palette, color):
    palette = np.array(palette)
    color = np.array(color)
    distances = np.sqrt(np.sum((palette-color)**2, axis=1))
    index_of_smallest = np.where(distances==np.amin(distances))
    smallest_distance = palette[index_of_smallest[0]][0]
    return smallest_distance 

def quantimage(image, palette):
    # Prepare for quantization
    rows, cols, channels = image.shape
    quantized = np.zeros_like(image)

    # Apply quantization
    for r in range(rows):
        for c in range(cols):
            # Extract the original pixel value
            pixel = image[r, c, :]
            # Find the closest colour from the palette (using absolute value/Euclidean distance)
            # Note: You may need more than one line of code here
            new_pixel = closest(palette, pixel)

            # Apply quantization
            quantized[r, c, :] = new_pixel
    return quantized

quantized_custom_4_colors = quantimage(img, colors)

In [ ]:
def normalize(image):
    return cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
# Show quantized image (don't forget to cast back to uint8)
plt.imshow(normalize(quantized_custom_4_colors))

In [ ]:
def mse(original, compressed):
    return np.mean((original - compressed) ** 2)

def psnr(original, compressed):
    mse_val = mse(original, compressed)
    if(mse_val == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse_val))
    return psnr

In [ ]:
# Compute average quantization error
quantized_image_4_mse = mse(img_norm, normalize(quantized_custom_4_colors))
quantized_image_4_psnr = psnr(img_norm, normalize(quantized_custom_4_colors))

#### Floyd-Steinberg Dithering
We are now going to implement the FS dithering and compare it to the optimally quantized image we have calculated above.

In [ ]:
def floyd_steinberg_dithering(image, palette):
    rows, cols, channels = image.shape
    # Make a temporal copy of the original image, we will need it for error diffusion
    img_tmp = np.copy(image)
    dithering = np.zeros_like(image)

    for r in range(1, rows-1):
        for c in range(1, cols-1):
            # Extract the original pixel value
            pixel = img_tmp[r, c, :]
            # Find the closest colour from the palette (using absolute value/Euclidean distance)
            # Note: You may need more than one line of code here
            new_pixel = closest(palette, pixel)
            
            # Compute quantization error
            quant_error = pixel - new_pixel
            
            # Diffuse the quantization error accroding to the FS diffusion matrix
            # Note: You may need more than one line of code here
            img_tmp[r, c, :] = new_pixel
            img_tmp[r + 1, c, :] = img_tmp[r + 1, c, :] + quant_error * 7 / 16
            img_tmp[r - 1, c + 1, :] = img_tmp[r - 1, c + 1, :] + quant_error * 3 / 16
            img_tmp[r, c + 1, :] = img_tmp[r, c + 1, :] + quant_error * 5 / 16
            img_tmp[r + 1, c + 1, :] = img_tmp[r + 1, c + 1, :] + quant_error * 1 / 16
            
            # Apply dithering
            dithering[r, c, :] = new_pixel
    return dithering

dithering_custom_4_colors = floyd_steinberg_dithering(img, colors)

In [ ]:
# Show quantized image (don't forget to cast back to uint8)
plt.subplot(121), plt.imshow(quantized_custom_4_colors), plt.title('Optimally quantized')   # optimally quantized
plt.subplot(122), plt.imshow(dithering_custom_4_colors), plt.title('FS dithering')   # dithering

In [ ]:
# Compute average quantization error for dithered image
FS_dithering_4_mse = mse(img_norm, normalize(dithering_custom_4_colors))
FS_dithering_4_psnr = psnr(img_norm, normalize(dithering_custom_4_colors))

In [ ]:
print("MSE for image after quantization: {}\nMSE for image after fs-dithering: {}".format(quantized_image_4_mse, FS_dithering_4_mse))
print("PSNR for image after quantization: {}\nPSNR for image after fs-dithering: {}".format(quantized_image_4_psnr, FS_dithering_4_psnr))

### Questions
* Which image has higher quantization error? Optimally quantized or dithered?
    
    Dithered image has higher quantization error then optimally quantized

* Which image looks better to you?
    
    For me dithered image looks better. I can see more details on it.

* Can you repeat the same process using only two colours: black and white? Show me :-)
    
    Results images you can see in README.md in this homework. (Paragraph 2)

In [ ]:
black_white_palette = np.array([[0, 0, 0],
                                [255, 255, 255]])
black_white_palette = black_white_palette/255

In [ ]:
q_b_w_img = quantimage(img, black_white_palette)
d_b_w_img = floyd_steinberg_dithering(img, black_white_palette)

q_b_w_img_norm = normalize(q_b_w_img)
d_b_w_img_norm = normalize(d_b_w_img)

quantized_image_2_mse = mse(img_norm, q_b_w_img_norm)
quantized_image_2_psnr = psnr(img_norm, q_b_w_img_norm)
FS_dithering_2_mse = mse(img_norm, d_b_w_img_norm)
FS_dithering_2_psnr = psnr(img_norm, d_b_w_img_norm)

print("MSE for image after quantization: {}\nMSE for image after fs-dithering: {}".format(quantized_image_2_mse, FS_dithering_2_mse))
print("PSNR for image after quantization: {}\nPSNR for image after fs-dithering: {}".format(quantized_image_2_psnr, FS_dithering_2_psnr))

plt.subplot(121), plt.imshow(q_b_w_img), plt.title('Optimally quantized')
plt.subplot(122), plt.imshow(d_b_w_img), plt.title('FS dithering') 

In [ ]:
plt.subplot(121), plt.imshow(dithering_custom_4_colors[200:300,500:600]), plt.title('Dithering with 4 colors')
plt.subplot(122), plt.imshow(d_b_w_img[200:300,500:600]), plt.title('Dithering with 2 colors')

### Bonus Points

Repeat the homework using a diffrerent image palette. For instance, you can use an optimal colour
palette that we can calculate via k-means algorithm. The following snippet of code will give you the 16
optimal colours for your original image.

In [ ]:
from sklearn.cluster import KMeans
from showlist import show_image_list

LABALE = "Collors: {}, MSE: {:.2f}, PSNR: {:.2f}"

In [ ]:
#Init palletts

def get_collors(image, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters).fit(np.reshape(image, (-1, 3)))
    return kmeans.cluster_centers_

colors_4 = get_collors(img,4)
colors_8 = get_collors(img,8)
colors_16 = get_collors(img,16)
colors_32 = get_collors(img,32)
colors_256 = get_collors(img,256)

In [ ]:
#Quantize images and show result
q_4 = quantimage(img, colors_4)
q_8 = quantimage(img, colors_8)
q_16 = quantimage(img, colors_16)
q_32 = quantimage(img, colors_32)
q_256 = quantimage(img, colors_256)

q_images = [img, q_4, q_8, q_16, q_32, q_256]
q_lables = ['original', 
        LABALE.format(4, mse(img_norm, normalize(q_4)), psnr(img_norm, normalize(q_4))),
        LABALE.format(8, mse(img_norm, normalize(q_8)), psnr(img_norm, normalize(q_8))),
        LABALE.format(16, mse(img_norm, normalize(q_16)), psnr(img_norm, normalize(q_16))),
        LABALE.format(32, mse(img_norm, normalize(q_32)), psnr(img_norm, normalize(q_32))),
        LABALE.format(256, mse(img_norm, normalize(q_256)), psnr(img_norm, normalize(q_256)))
        ]

show_image_list(q_images, q_lables, grid=False, num_cols=3)

In [ ]:
#FS dithering images and show result
d_4 = floyd_steinberg_dithering(img, colors_4)
d_8 = floyd_steinberg_dithering(img, colors_8)
d_16 = floyd_steinberg_dithering(img, colors_16)
d_32 = floyd_steinberg_dithering(img, colors_32)
d_256 = floyd_steinberg_dithering(img, colors_256)

d_images = [img, d_4, d_8, d_16, d_32, d_256]
d_lables = ['original', 
        LABALE.format(4, mse(img_norm, normalize(d_4)), psnr(img_norm, normalize(d_4))),
        LABALE.format(8, mse(img_norm, normalize(d_8)), psnr(img_norm, normalize(d_8))),
        LABALE.format(16, mse(img_norm, normalize(d_16)), psnr(img_norm, normalize(d_16))),
        LABALE.format(32, mse(img_norm, normalize(d_32)), psnr(img_norm, normalize(d_32))),
        LABALE.format(256, mse(img_norm, normalize(d_256)), psnr(img_norm, normalize(d_256)))
        ]

show_image_list(d_images, d_lables, grid=False, num_cols=3)

Apply FS dithering the same way you did before.
* How does the result look like to you?
    
    The results looks better than from palette with gray colors. From 16 colors in palette visual difference from original image is not so noticeable.

* What happens if we use 32 colours?

    MSE gets smaller and PSNR gets bigger. Background look like has some noize.

* And what happens if we use 256 colours?

    MSE gets smaller and PSNR gets bigger. Image looks like original

Result images in README.md file in this homework. (Paragraph 3 and 4)